In [1]:
!pip install -r requirement.txt

In [2]:
import numpy as np
import geatpy as ea
import pandas as pd
import random
from datetime import datetime
import parser
print(ea.__version__)

debug = 1 # debug mode

2.7.0


### READ DATA
TODO: read data from GP

In [3]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')

In [4]:
# randomly choose 1000 samples
n = 10000
rawData = rawData.iloc[:30000, :61]

In [5]:
data = rawData.sample(n)
print(data.shape)
data.sample(3)

(10000, 61)


,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,...,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units
10715,FZOPS,DF3RM,FFAU3660891,4420.0,DFIC-FL-2104,3/24/2021,1698636.0,DFC3,D4H,FFAU3660891,...,1.7,2021 NP,Reasonable,3.435164,10.423562,from ANZ portfolio,LE-ONE-09AD4H,NaN,2000.0,1.0
14912,FZOPS,DF3RM,FFAU4037232,6000.0,CSIH-FL-2115,4/8/2021,1706620.0,DFC3,D4H,FFAU4037232,...,1.7,2021 NP,Reasonable,3.367164,10.637808,from ANZ portfolio,LT-MSC-48D4H,NaN,6997.0,1.0
24871,FZOPS,DF3RM,FBIU0353710,2267.0,CIMC-FL-1804,3/22/2018,1185717.0,DFC3,D20,FBIU0353710,...,1.0,2018 NP,Reasonable,5.020685,3.545205,from available portfolio,LE-ONE-01D20,NaN,4991.0,1.0


### DATA PREPARATION

In [6]:
# TODO: add more columns later
input_data = \
    data.loc[:, ['Cost', 'Mfr Dt Fz', 'Product', 'Contract Cust Id', 'Contract Dt', 'Contract Expiration Dt', 'Nbv', 'Billing Status Fz']]
input_data.head(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
17002,6000.0,5/9/2021,D4H,MSC,2021-04-01,2029-06-30,5612.350,ON
12882,4420.0,3/3/2021,D4H,ONE,2020-11-16,2029-05-15,4117.430,ON
22195,2267.0,3/22/2018,D20,ONE,2018-02-01,2026-10-14,1888.001,ON


#### Find standard equipment

Assign to `Product`

In [7]:
# select standard equipment
standardEquipment = ['D4H', 'D20', 'D40']
def SelectStandardEquip(Product):
    if Product in standardEquipment:
        Product = 1
    else:
        Product = 0
    return Product

input_data['Product'] = input_data.apply(lambda x: SelectStandardEquip(x['Product']), axis=1)
input_data.head(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
17002,6000.0,5/9/2021,1,MSC,2021-04-01,2029-06-30,5612.350,ON
12882,4420.0,3/3/2021,1,ONE,2020-11-16,2029-05-15,4117.430,ON
22195,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1888.001,ON


#### Find Top 1 and Top 3 Lessees

Assign new column `Top1Lessee` and `Top3Lessees`

In [8]:
# select the Top1 lessee
input_data.groupby('Contract Cust Id').count().sort_values('Cost', ascending=False)

,Cost,Mfr Dt Fz,Product,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
Contract Cust Id,,,,,,,
MSC,4152,4152,4152,4152,4152,4152,4152
ONE,3711,3711,3711,3711,3711,3711,3711
HAPAG,1950,1950,1950,1950,1950,1950,1950
TCLC,67,67,67,67,67,67,67
COSMR,48,48,48,48,48,48,48
CMA,42,42,42,42,42,42,42
PANOCEAN,30,30,30,30,30,30,30


In [9]:
# top1 and top3 lessees 
Top1Lessee = 'MSC'
Top3Lessees = ['MSC', 'ONE', 'HAPAG']

In [10]:
def SelectTop1(custID):
    if custID == Top1Lessee:
        custID = 1
    else:
        custID = 0
    return custID

def SelectTop3(custID):
    if custID in Top3Lessees:
        custID = 1
    else:
        custID = 0
    return custID

input_data['Top1Lessee'] = input_data.apply(lambda x: SelectTop1(x['Contract Cust Id']), axis=1)
input_data['Top3Lessees'] = input_data.apply(lambda x: SelectTop3(x['Contract Cust Id']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees
1164,3832.0,12/18/2017,1,HAPAG,2017-11-01,2030-01-31,2981.838,ON,0,1
23216,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1896.090,ON,0,1
17195,6000.0,5/9/2021,1,MSC,2021-04-01,2029-06-30,5612.350,ON,1,1


#### Calcualte manufacture age
Assign new column `MfrAge`

In [11]:
# TODO: consider timezone, consider time_now in the future
time_now = datetime.now()
# calculate manufacture age
date_format = '%m/%d/%Y'
# return in years, round to 2 decimal
def CalculateManufacutreAge(date):
    diff = time_now.date() - datetime.strptime(date, date_format).date()
    return round(diff.days / 365, 2)

input_data['MfrAge'] = input_data.apply(lambda x: CalculateManufacutreAge(x['Mfr Dt Fz']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge
7515,3702.0,6/11/2018,1,MSC,2018-03-01,2032-07-31,2983.22,ON,1,1,4.30
15064,6000.0,4/8/2021,1,MSC,2021-04-01,2029-06-30,5612.35,ON,1,1,1.47
15842,6000.0,4/8/2021,1,MSC,2021-04-01,2029-06-30,5612.35,ON,1,1,1.47


#### Calculate unexpired year 
Assign new column `ExpYear`

In [12]:
# TODO: consider timezone, consider time_now in the future
# TODO: 'Contract Expiration Dt' is in Timestamp, the data type may change
time_now = datetime.now()
# calculate manufacture age
date_format = '%Y-%m-%d'
# return in years, round to 2 decimal
def CalculateExpYear(date):
    diff = pd.to_datetime(date,format="%Y%m%d").date() - time_now.date()
    return round(diff.days / 365, 2)

input_data['ExpYear'] = input_data.apply(lambda x: CalculateExpYear(x['Contract Expiration Dt']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
691,3832.0,12/18/2017,1,HAPAG,2017-11-01,2030-01-31,2981.836,ON,0,1,4.78,7.35
11361,4420.0,3/24/2021,1,ONE,2020-11-16,2029-05-15,4155.250,ON,0,1,1.51,6.64
9054,5150.0,4/20/2021,1,MSC,2021-04-01,2029-06-30,4773.960,ON,1,1,1.44,6.76


Select billing status

In [13]:
def SelectBilling(status):
    if status == 'ON':
        status = 1
    else:
        status = 0
    return status

input_data['Billing Status Fz'] = input_data.apply(lambda x: SelectBilling(x['Billing Status Fz']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
22219,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1888.001,1,0,1,4.52,4.05
24891,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1880.705,1,0,1,4.52,4.05
21291,3849.0,4/6/2018,1,ONE,2018-02-01,2026-10-14,3123.456,1,0,1,4.48,4.05


Save data

In [14]:
input_data.to_csv('prepared_data.csv')

In [15]:
prepared_data = pd.read_csv('./prepared_data.csv')
prepared_data.head(3)

,Unnamed: 0,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
0,17002,6000.0,5/9/2021,1,MSC,2021-04-01,2029-06-30,5612.350,1,1,1,1.38,6.76
1,12882,4420.0,3/3/2021,1,ONE,2020-11-16,2029-05-15,4117.430,1,0,1,1.57,6.64
2,22195,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1888.001,1,0,1,4.52,4.05


### Model Part

#### Input

Convert pandas to numpy array

TODO: set input, may consider alt way.

In [16]:
prepared_data.describe()

,Unnamed: 0,Cost,Product,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
count,10000.000000,10000.000000,10000.0,10000.000000,10000.0,10000.000000,10000.00000,10000.000000,10000.000000
mean,15018.425400,3980.591125,1.0,3436.744091,1.0,0.415200,0.98130,3.523284,6.694100
std,8694.542765,1232.768322,0.0,1283.384989,0.0,0.492781,0.13547,1.490244,2.028959
min,2.000000,2182.000000,1.0,1784.484000,1.0,0.000000,0.00000,1.350000,1.010000
25%,7456.000000,3682.000000,1.0,2869.540000,1.0,0.000000,1.00000,1.510000,4.050000
50%,15101.500000,3832.000000,1.0,2981.834000,1.0,0.000000,1.00000,4.520000,6.760000
75%,22581.750000,4420.000000,1.0,4136.340000,1.0,1.000000,1.00000,4.730000,7.350000
max,29999.000000,7081.000000,1.0,6626.060000,1.0,1.000000,1.00000,4.840000,9.850000


In [17]:
cost = prepared_data['Cost'].to_numpy().reshape(n, 1)
nbv = prepared_data['Nbv'].to_numpy().reshape(n, 1)
product = prepared_data['Product'].to_numpy()
mfrAge = prepared_data['MfrAge'].to_numpy().reshape(n, 1)
lesseStatus = prepared_data['Billing Status Fz'].to_numpy()
expYear = prepared_data['ExpYear'].to_numpy().reshape(n, 1)
top1 = prepared_data['Top1Lessee'].to_numpy()
top3 = prepared_data['Top3Lessees'].to_numpy()

if debug:
    print('nbv shape: ', nbv.shape)
    print('product shape: ', product.shape)
    print('mfrAge shape: ', mfrAge.shape)
    print('lesseStatus shape: ', lesseStatus.shape)
    print('expYear shape: ', expYear.shape)


nbv shape:  (10000, 1)
product shape:  (10000,)
mfrAge shape:  (10000, 1)
lesseStatus shape:  (10000,)
expYear shape:  (10000, 1)


#### Constraints

`x`: np.array, size=num_of_container. `x[i]=1` stands for $i^{th}$ container is selected; `x[i]=0` stands for $i^{th}$ container is not selected.

1. NBV of standard equip >= 80%: `sum(x*Product*Nbv) / sum(x*Nbv) >= 0.8`

2. Avg age of equip <= 3.75: TODO

3. Avg age of Manufacturing < 7: `sum(x*MfrAge) / sum(x) <= 7`

4. Equipment port subject to lease >= 90%: TODO

5. Avg unexpired term under leases >= 5: `sum(x*Billing Status Fz*ExpYear) / sum(x*Billing Status Fz) >= 5`

6. Top 1 NBV <= 50%: `sum(x*Top1Lessee*Nbv) / sum(x*Nbv) <= 0.5`

7. Top 3 NBV <= 70%: `sum(x*Top3Lessee*Nbv) / sum(x*Nbv) <= 0.7`

#### Objective

min Cost: `min sum(x*Cost)`

#### Define Problem
Define objective function and constraint functions

In [27]:
def DEMO():
    class MyProblem(ea.Problem):
        def __init__(self):
            name ='MyProblem'
            M = 1
            maxormins = [1] # -1: max, 1: min
            Dim = n # num of variables
            varTypes = [1] * Dim # integers: 0 or 1
            lb = [0] * Dim # low bound
            ub = [1] * Dim # up bound
            lbin = [1] * Dim # low bound include
            ubin = [1] * Dim # up bound include
            ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb,ub, lbin, ubin)
            
        def aimFunc(self, pop):
            Vars = pop.Phen
            
            # two optimization methods are showed below. now we only care the first one.
            if (1):
                pop.ObjV = Vars @ cost
                
                pop.CV = np.hstack([
                    -((Vars * product @ nbv) / (Vars @ nbv) - 0.8),
                    (Vars @ mfrAge) / Vars.sum(axis=1, keepdims=True) - 7,
                    -((Vars * lesseStatus @ expYear) / (Vars * lesseStatus).sum(axis=1, keepdims=True) - 5),
#                     (Vars * top1 @ nbv) / (Vars @ nbv) - 0.5,
#                     (Vars * top3 @ nbv) / (Vars @ nbv) - 0.7
                ])

            else:
                # convert constraint problem to no-constaint problem
                # objective function
                f = 4*x1 + 2*x2 + x3
                # find the index of instances that violate constraints
                exIdx1 = np.where(2*x1 + x2 > 1)[0]
                exIdx2 = np.where(x1 + 2*x3 > 2)[0]
                exIdx3 = np.where(x1 + x2 + x3 != 1)[0]
                exIdx = np.unique(np.hstack([exIdx1, exIdx2, exIdx3]))
                alpha = 2
                beta = 1
                f[exIdx] += self.maxormins[0] * alpha * (np.max(f) - np.min(f) + beta)
                # set objective function
                pop.ObjV = f 


    problem = MyProblem()
    Encoding ='RI'
    NIND = 1000 # init number of population instance
    Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges,problem.borders)
    population = ea.Population(Encoding, Field, NIND)

    # set parameters
    myAlgorithm = ea.soea_DE_best_1_L_templet(problem, population)
    myAlgorithm.MAXGEN = 10000
    myAlgorithm.mutOper.F = 0.5
    myAlgorithm.recOper.XOVR = 0.7
    myAlgorithm.logTras = 1
#     myAlgorithm.trappedValue = 1e-6
#     myAlgorithm.maxTrappedCount = 100

    # TODO: random seed has a significant influence on final result!
    # outputMsg=True to show the result
    res = ea.optimize(myAlgorithm, seed=2049, verbose=False, drawing=1, outputMsg=True, drawLog=False, saveFlag=False, dirName='result')
    if debug:
        print(res)
    return res

In [ ]:
# run and debug
selectedPackage = DEMO()

#### Output

In [ ]:
print('Time Consumption: ', np.round(selectedPackage['executeTime'], decimals=2), 's')
print(selectedPackage['stopMsg'])

if selectedPackage['success']:
    print('Algorithm Succeeded!')
    print('===================================================================================================')
    print('Optimal Target Value: ', np.round(selectedPackage['ObjV'].item(), decimals=2))
    print('Constraint Violation: ', np.round(selectedPackage['CV'][0], decimals=2))
    if debug:
        # TODO: doing
        print('===================================================================================================')
        Vars = selectedPackage['Vars'][0]
        print('NBV of standard equip:', np.round((Vars * product @ nbv) / (Vars @ nbv), decimals=2)[0])
        print('Avg age of Manufacturing:', np.round((Vars @ mfrAge) / Vars.sum(), decimals=2)[0])
        print('Avg unexpired term under leases:', np.round((Vars * lesseStatus @ expYear) / (Vars * lesseStatus).sum(), decimals=2)[0])
        print('Top 1 NBV:', np.round((Vars * top1 @ nbv) / (Vars @ nbv), decimals=2)[0])
        print('Top 3 NBV:', np.round((Vars * top3 @ nbv) / (Vars @ nbv), decimals=2)[0])
        
else:
    print('No Feasible Results...')

In [23]:
# print final result
print('Selected Package:')
print('===================================================================================================')
print(len(selectedPackage['Vars'].nonzero()[1]), 'containers selected.')
print(selectedPackage['Vars'].nonzero()[1]) # index begins from 0
print('===================================================================================================')
print(selectedPackage['Vars'][0].tolist())

Selected Package:
4890 containers selected.
[   0    2    3 ... 9994 9995 9997]
[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1